In [23]:
news_texts="""Giant stone head resembling an ancient Indigenous sculpture sits on top of a crushed Tesla car. Look what I do to your lousy car with this wonderful head. This is bigger than you and the rampant technologies, the sculptor says. A video released by the Colima 71 hotel shows the moment when the head was released and the car's roof gradually caved in. We wanted to preserve the magic and mystery of how it arrived, to let the work itself generate an impact and spark conversations, the hotel's artistic director says. It's the wonderful thing about art, it allows you these atrocities, says the 42-year-old sculptor. The head was inspired by the colossal head carvings of the Olmec, considered the first known major Mesoamerican civilization. The electric carmaker announced plans to build a huge factory in northern Mexico a year after announcing plans to sell cars in the U.S. and China.. The car's batteries had been removed before its roof was crushed.
” 
"""

In [4]:
import os
import requests
import yfinance as yf
from textblob import TextBlob
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from scipy.special import softmax
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import BartTokenizer, BartForConditionalGeneration

In [24]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
sia=SentimentIntensityAnalyzer()
sia.polarity_scores(news_texts)


{'neg': 0.052, 'neu': 0.845, 'pos': 0.103, 'compound': 0.8338}

In [25]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [26]:
model_name="cardiffnlp/twitter-roberta-base-sentiment"
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForSequenceClassification.from_pretrained(model_name)

In [27]:
encoded=tokenizer(news_texts,return_tensors='pt')
output=model(**encoded)
scores=output[0][0].detach().numpy()
scores=softmax(scores)
print(scores)


[0.0647918  0.35041124 0.58479697]


In [28]:
weights = {
    'neg': -1,
    'neu': 0,
    'pos': 1
}
probabilities = {
    'neg': scores[0],
    'neu': scores[1],
    'pos': scores[2]
}
compound_score = sum(probabilities[label] * weights[label] for label in probabilities)
print(f"Compound score: {compound_score}")

Compound score: 0.5200051665306091


In [13]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def find_url(keyword):
    # Convert dates to the format Google expects (M/D/YYYY)
    start_date = "6/8/2023"
    end_date = "6/8/2023"

    search_query = keyword.replace(" ", "+")
    # Construct the URL with the tbs parameter for date range
    link = f"https://www.google.com/search?q={search_query}&tbs=cdr%3A1%2Ccd_min%3A{start_date[0]}%2F{start_date[2]}%2F{start_date[4:]}%2Ccd_max%3A{start_date[0]}%2F{start_date[2]}%2F{start_date[4:]}&tbm=nws"
    print(link)
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(link, headers=headers)
    webpage = response.content
    soup = BeautifulSoup(webpage, 'html5lib')
    links = []
    for div_tag in soup.find_all('div', class_='Gx5Zad'):
        a_tag = div_tag.find('a')
        if a_tag and 'href' in a_tag.attrs:
            href = a_tag['href']
            if href.startswith('/url?q='):
                url = href.split('/url?q=')[1].split('&sa=')[0]
                links.append(url)
    return links

# Example usage
keyword = "elon musk"
start_date = "2024-06-04"
end_date = "2024-06-04"
urls = find_url(keyword)
for url in urls:
    print(url)


https://www.google.com/search?q=elon+musk&tbs=cdr%3A1%2Ccd_min%3A6%2F8%2F2023%2Ccd_max%3A6%2F8%2F2023&tbm=nws


In [23]:
def find_url(keyword):
    start_date = "6/8/2023"    
    root = "https://www.google.com/"
    search_query = keyword.replace(" ", "+")
    link = f"https://www.google.com/search?q={search_query}&tbs=cdr%3A1%2Ccd_min%3A{start_date[0]}%2F{start_date[2]}%2F{start_date[4:]}%2Ccd_max%3A{start_date[0]}%2F{start_date[2]}%2F{start_date[4:]}"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(link, headers=headers)
    webpage = response.content
    soup = BeautifulSoup(webpage, 'html5lib')
    links = []
    # print(soup.find_all('div'))
    # print(link)
    
    for div_tag in soup.find_all('div', class_='Gx5Zad'):
        a_tag = div_tag.find('a')
        if a_tag:
            if 'href' in a_tag.attrs:
                href = a_tag['href']
                if href.startswith('/url?q='):
                    url = href.split('/url?q=')[1].split('&sa=')[0]
                    links.append(url)
    return link

In [24]:
a=find_url("elon musk news")
print(a)

https://www.google.com/search?q=elon+musk+news&tbs=cdr%3A1%2Ccd_min%3A6%2F8%2F2023%2Ccd_max%3A6%2F8%2F2023


In [2]:
from datetime import datetime, timedelta
from exa_py import Exa

exa = Exa(api_key="9fccb5a3-0bff-453b-93bf-dd7e82f7a690")

one_week_ago = (datetime.now() - timedelta(days=7))
date_cutoff = one_week_ago.strftime("%Y-%m-%d")

search_query = "hdfc news "
search_response = exa.search_and_contents(
    search_query, use_autoprompt=True, start_published_date=date_cutoff,end_published_date=date_cutoff
)

urls = [result.url for result in search_response.results]
print("URLs:")
for url in urls:
    print(url)


# result = exa.search_and_contents(
#   "HDFC stock latest news in a week",
#   type="neural",
#   use_autoprompt=True,
#   num_results=10,
#   text=True
# )

results = search_response.results
result_item = results[0]
print(f"{len(results)} items total, printing the first one:")
print(result_item.text)

URLs:
https://www.cnbc.com/2024/06/17/-wealth-manager-for-the-ultra-rich-names-3-global-stocks-to-buy-.html?__source=newsletter%7Cmsemea
https://www.businesstoday.in/india/story/west-bengal-kanchanjunga-express-train-derailed-by-goods-train-in-bengals-siliguri-no-deaths-reported-so-far-mamata-banerjee-433512-2024-06-17
https://www.hindustantimes.com/business/zepto-to-diversify-from-groceries-into-higher-value-items-looks-to-set-up-bigger-outlets-101718617108573.html
https://www.rediff.com/business/report/sebi-signals-change-with-tougher-futures-and-options-stock-rules/20240617.htm
https://fuelcellsworks.com/news/essar-group-commits-rs-30000-crore-3-6-billion-to-pioneer-green-and-blue-hydrogen-projects-across-global-operations/?utm_source=rss&utm_medium=rss&utm_campaign=essar-group-commits-rs-30000-crore-3-6-billion-to-pioneer-green-and-blue-hydrogen-projects-across-global-operations
https://www.rediff.com/business/report/modi-30-railways-to-begin-hiring-again/20240617.htm
https://www.r

In [45]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote

def find_news_url(keyword, start_date, end_date):
    root = "https://www.google.com/"
    search_query = keyword.replace(" ", "+")
    link = f"{root}search?q={search_query}&tbm=nws&tbs=cdr:1,cd_min:{start_date},cd_max:{end_date}"
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    news_links = []
    
    for article in soup.select('div.SoaBEf'):
        link = article.select_one('a')
        if link and 'href' in link.attrs:
            url = link['href']
            if url.startswith('/url?q='):
                url = unquote(url.split('/url?q=')[1].split('&sa=')[0])
            news_links.append(url)
    
    return news_links

# Example usage
keyword = "elon musk"
start_date = "6/8/2023"
end_date = "6/8/2023"
results = find_news_url(keyword, start_date, end_date)
print(len(results))

10


In [61]:
from bs4 import BeautifulSoup
import requests
import yfinance as yf


In [62]:
def scraping_article(url):
    headers = {
    'User-Agent': 'Your User Agent String',
    }
    r=requests.get(url,headers=headers)
    soup=BeautifulSoup(r.text,'html.parser')
    paragraphs=soup.find_all('p')
    text= [paragraph.text for paragraph in paragraphs]
    words=' '.join(text).split(' ')
    article = ' '.join(words)
    return article

In [63]:
url="https://www.motor1.com/news/702440/tesla-plaid-vs-dodge-demon-170-drag-race/"
a=scraping_article(url)

In [64]:
print(a)

In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import spacy

In [16]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import spacy
import re

# Load spaCy for text processing
nlp = spacy.load("en_core_web_sm")

# Load RoBERTa sentiment analysis model
sentiment_analyzer = pipeline("sentiment-analysis", model="roberta-base")

def analyze_keyword_sentiment(text, keyword):
    # Normalize text and keyword
    text = text.lower()
    keyword = keyword.lower()

    # Check if keyword is in the text
    if keyword not in text:
        return "Neutral (Keyword not found)", 0

    # Tokenize the text
    doc = nlp(text)
    
    # Find sentences containing the keyword
    relevant_sentences = [sent.text for sent in doc.sents if keyword in sent.text.lower()]
    
    if not relevant_sentences:
        return "Neutral (Keyword not in complete sentence)", 0

    # Analyze sentiment of relevant sentences
    sentiments = [sentiment_analyzer(sent)[0] for sent in relevant_sentences]
    
    # Calculate weighted sentiment score
    total_score = sum(sentiment['score'] if sentiment['label'] == 'POSITIVE' else -sentiment['score'] for sentiment in sentiments)
    avg_score = total_score / len(sentiments)
    
    # Determine overall sentiment
    if avg_score > 0.1:
        label = "Positive"
    elif avg_score < -0.1:
        label = "Negative"
    else:
        label = "Neutral"

    return label, avg_score

# Example usage

keyword = "byd"

sentiment, score = analyze_keyword_sentiment(news_texts, keyword)
print(f"Sentiment for '{keyword}': {sentiment}")
print(f"Sentiment score: {score:.2f}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sentiment for 'byd': Negative
Sentiment score: -0.53
